<a href="https://colab.research.google.com/github/Albieri-Ray/Banco_BBVA/blob/main/Parrillas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parrilla Incentivación

Cargar la información en el dataframe

In [ ]:
import pandas as pd

# Carga del archivo Excel "Parrilla.xlsx"
parrilla_df = pd.read_excel('/content/Parrilla.xlsx', sheet_name='parrilla')
extrabono_df = pd.read_excel('/content/Parrilla.xlsx', sheet_name='extrabono')

# Renombrar las columnas para que coincidan la estructura
parrilla_df.columns = [col.replace(" ", "_").upper() for col in parrilla_df.columns]
extrabono_df.columns = [col.replace(" ", "_").upper() for col in extrabono_df.columns]

# Crear la tabla temporal TMP_PARRILLA
tmp_parrilla_df = parrilla_df.melt(id_vars=['N', 'INDICADOR', 'NOMENCLATURA'],
                                   var_name='PERFIL', value_name='PESO')

# Filtrar filas con valores nulos en PESO
tmp_parrilla_df = tmp_parrilla_df.dropna(subset=['PESO'])

# Creando la columna Periodo con parametro
tmp_parrilla_df['PERIODO'] = '202406'  # Utilizar un valor fijo para PERIODO, esto puede ser parametrizado

# Asignar un valor de 99 al orden cuando el peso sea 0, y el valor de N en caso contrario
tmp_parrilla_df['ORDEN_PESO'] = tmp_parrilla_df.apply(lambda row: 99 if row['PESO'] == 0 else row['N'], axis=1)

# Ordenar los datos por PERFIL y ORDEN_PESO (los de peso 0 al final)
tmp_parrilla_df = tmp_parrilla_df.sort_values(by=['ORDEN_PESO', 'PESO'], ascending=[True, False])

# Asignar el orden consecutivo por perfil
tmp_parrilla_df['ORDEN'] = tmp_parrilla_df.groupby('PERFIL').cumcount() + 1

# Función de la columna TIPO
def tipo_indicador(indicador, perfil):
    if indicador in ['CA', 'MIGTRA', 'CAPYMEOF', 'CAPYME', 'CAPN','TNMONE']:
        return 'I'
    elif indicador == 'TESP':
        return 'T'
    elif indicador in ['PARF', 'FUNICO', 'CSAM']:
        return 'S'
    elif indicador == 'EU':
        return 'B1'
    elif indicador in ['CAPEJE', 'CAPEJECORE', 'CAPOFI', 'CAPOFIEJEC', 'QUIADI', 'ROTAS'] or (indicador == 'CAPAS' and (perfil == 'SGO3' or perfil.startswith('GOF'))):
        return 'B2'
    elif indicador in ['PENCA','B_CONS','B_PYME']:
        return 'B3'
    elif indicador == 'QDACHMP':
        return 'B5'
    else:
        return 'D'

tmp_parrilla_df['TIPO'] = tmp_parrilla_df.apply(lambda row: tipo_indicador(row['NOMENCLATURA'], row['PERFIL']), axis=1)

# Merge con la tabla de extrabono para la salida final
parrilla_bcom_df = tmp_parrilla_df.merge(extrabono_df, left_on=['PERFIL', 'NOMENCLATURA'], right_on=['PARRILLA', 'NOMENCLATURA'], how='left')

def tipo_umbral(indicador, umbral1):
    umbral1 = umbral1 if not pd.isna(umbral1) else 0
    if indicador == 'VINTAR' and (umbral1 < 100 and umbral1 > 0):
        return 'TRN'
    elif indicador == 'RTT':
        if umbral1 < 30 and umbral1 > 0:
            return 'TRN2'
        else:
            return 'POR'
    elif indicador == 'FTHIP' and umbral1 > 0:
        return 'TRN3'
    elif indicador.startswith('NPOS') and (umbral1 < 100 and umbral1 > 0):
        return 'RES'
    elif indicador == 'QUINAHO' and umbral1 != 0:
        return 'NRO'
    elif umbral1 < 100 and umbral1 > 0:
        return 'NRO'
    elif umbral1 >= 100:
        return 'POR'
    else:
        return ''

parrilla_bcom_df['TIPO_UMBRAL'] = parrilla_bcom_df.apply(lambda row: tipo_umbral(row['NOMENCLATURA'], row['UMBRAL1']), axis=1)

# Añadir las columnas faltantes con valores flat
parrilla_bcom_df['CORE'] = 'NO'
parrilla_bcom_df['F1'] = 0.9

# Lógica para actualizar la columna CORE
core_pj = ['FACTVAPENPNNPJ','VARSPVAPJOFIPEN','FTPYME','NTKT','SEG','STOCKNOM','NPOSEBN','FACTVAPNNPJ','VARSMVAPJPENDOL','NCLNUE']
parrilla_bcom_df.loc[(parrilla_bcom_df['PERFIL'].str.contains('EBN')) & (parrilla_bcom_df['NOMENCLATURA'].isin(core_pj)), 'CORE'] = 'SI'

core_pn = ['FACTFFMMPLZ','FTPLDVEH','NTC','SEG','SPVAAS','FTCNVA']
parrilla_bcom_df.loc[(parrilla_bcom_df['PERFIL'].str.contains('EBP') | parrilla_bcom_df['PERFIL'].str.contains('AS')) & (parrilla_bcom_df['NOMENCLATURA'].isin(core_pn)), 'CORE'] = 'SI'

# Reemplazar valores NaN en la columna 'INDICADOR' con '-'
parrilla_bcom_df.loc[parrilla_bcom_df['INDICADOR'].str.startswith(' '), 'INDICADOR'] = '- ' + parrilla_bcom_df['INDICADOR'].str.strip()

# Reemplazar guiones bajos por espacios en el campo PERFIL
parrilla_bcom_df['PERFIL'] = parrilla_bcom_df['PERFIL'].str.replace('_', ' ')

# Ordenar las columnas con el formato
parrilla_bcom_final = parrilla_bcom_df[['PERIODO', 'PERFIL', 'INDICADOR', 'NOMENCLATURA', 'PESO', 'ORDEN', 'TIPO', 'TIPO_UMBRAL',
                                        'UMBRAL1', 'UMBRAL2', 'UMBRAL3', 'UMBRAL4', 'PREMIO1', 'PREMIO2', 'PREMIO3', 'PREMIO4', 'TRAMO',
                                        'MAXIMO', 'CORE', 'F1']]


# Guarda el dataframe en un archivo Excel
salida_parrilla = '/content/Parrilla_BCOM_Final.xlsx'
parrilla_bcom_final.to_excel(salida_parrilla, index=False)

salida_parrilla


'/content/Parrilla_BCOM_Final.xlsx'